In [1]:
import selenium
import pandas as pd
import torch
from transformers import pipeline
import transformers
import numpy as np
import spacy
from spacy import displacy

D:\Conda\envs\FT\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [2]:
assert torch.cuda.is_available()

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
nltk.download('wordnet')  #download if using this module for the first time
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')    #download if using this module for the first time
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, pos_tag, ne_chunk
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\siddi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\siddi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\siddi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\siddi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\siddi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\siddi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker.zip.


True

In [5]:
df = pd.read_csv("../data/skills.csv")
df.head()

,Unnamed: 0,skills,summary
0,0,Deep Learning,Deep learning (also known as deep structured ...
1,1,Convolutional Neural Networks,"In deep learning, a convolutional neural netwo..."
2,2,TensorFlow,TensorFlow is a free and open-source software ...
3,3,Machine Learning,Machine learning (ML) is the study of computer...
4,4,Algorithms,"In mathematics and computer science, an algori..."


In [6]:
textexample = df.summary[0]
print(textexample)

Deep learning  (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised.Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks and convolutional neural networks have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.Artificial neural networks (ANNs) were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains.  Specifically, artificial neural networks tend to be static and symbolic, while th

In [7]:
sentences = nltk.sent_tokenize(textexample)
tokenized_sentence = [nltk.word_tokenize(sent) for sent in sentences]
tokenized_sentence 

[['Deep',
  'learning',
  '(',
  'also',
  'known',
  'as',
  'deep',
  'structured',
  'learning',
  ')',
  'is',
  'part',
  'of',
  'a',
  'broader',
  'family',
  'of',
  'machine',
  'learning',
  'methods',
  'based',
  'on',
  'artificial',
  'neural',
  'networks',
  'with',
  'representation',
  'learning',
  '.'],
 ['Learning',
  'can',
  'be',
  'supervised',
  ',',
  'semi-supervised',
  'or',
  'unsupervised.Deep-learning',
  'architectures',
  'such',
  'as',
  'deep',
  'neural',
  'networks',
  ',',
  'deep',
  'belief',
  'networks',
  ',',
  'deep',
  'reinforcement',
  'learning',
  ',',
  'recurrent',
  'neural',
  'networks',
  'and',
  'convolutional',
  'neural',
  'networks',
  'have',
  'been',
  'applied',
  'to',
  'fields',
  'including',
  'computer',
  'vision',
  ',',
  'speech',
  'recognition',
  ',',
  'natural',
  'language',
  'processing',
  ',',
  'machine',
  'translation',
  ',',
  'bioinformatics',
  ',',
  'drug',
  'design',
  ',',
  'medical'

In [8]:
pos_tagging_sentences = [nltk.pos_tag(sent) for sent in tokenized_sentence]

In [9]:
def preprocess(text):
    text = nltk.word_tokenize(text)
    text = nltk.pos_tag(text)
    return text

processed_text = preprocess(textexample)
processed_text

[('Deep', 'JJ'),
 ('learning', 'NN'),
 ('(', '('),
 ('also', 'RB'),
 ('known', 'VBN'),
 ('as', 'IN'),
 ('deep', 'JJ'),
 ('structured', 'VBD'),
 ('learning', 'NN'),
 (')', ')'),
 ('is', 'VBZ'),
 ('part', 'NN'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('broader', 'JJR'),
 ('family', 'NN'),
 ('of', 'IN'),
 ('machine', 'NN'),
 ('learning', 'VBG'),
 ('methods', 'NNS'),
 ('based', 'VBN'),
 ('on', 'IN'),
 ('artificial', 'JJ'),
 ('neural', 'JJ'),
 ('networks', 'NNS'),
 ('with', 'IN'),
 ('representation', 'NN'),
 ('learning', 'NN'),
 ('.', '.'),
 ('Learning', 'VBG'),
 ('can', 'MD'),
 ('be', 'VB'),
 ('supervised', 'VBN'),
 (',', ','),
 ('semi-supervised', 'JJ'),
 ('or', 'CC'),
 ('unsupervised.Deep-learning', 'JJ'),
 ('architectures', 'NNS'),
 ('such', 'JJ'),
 ('as', 'IN'),
 ('deep', 'JJ'),
 ('neural', 'JJ'),
 ('networks', 'NNS'),
 (',', ','),
 ('deep', 'JJ'),
 ('belief', 'NN'),
 ('networks', 'NNS'),
 (',', ','),
 ('deep', 'JJ'),
 ('reinforcement', 'NN'),
 ('learning', 'NN'),
 (',', ','),
 ('recurrent', 'J

In [10]:
res_chunk = ne_chunk(processed_text)

for x in str(res_chunk).split('\n'):
    if '/NN' in x:
        print(x)

  learning/NN
  learning/NN
  part/NN
  family/NN
  machine/NN
  methods/NNS
  networks/NNS
  representation/NN
  learning/NN
  architectures/NNS
  networks/NNS
  belief/NN
  networks/NNS
  reinforcement/NN
  learning/NN
  networks/NNS
  networks/NNS
  fields/NNS
  computer/NN
  vision/NN
  speech/NN
  recognition/NN
  language/NN
  processing/NN
  machine/NN
  translation/NN
  bioinformatics/NNS
  drug/NN
  design/NN
  image/NN
  analysis/NN
  inspection/NN
  board/NN
  game/NN
  programs/NNS
  results/NNS
  cases/NNS
  networks/NNS
  (ORGANIZATION ANNs/NNP)
  information/NN
  processing/NN
  communication/NN
  nodes/NNS
  systems/NNS
  ANNs/NNP
  differences/NNS
  brains/NNS
  networks/NNS
  brain/NN
  organisms/NNS
  plastic/NN
  learning/NN
  refers/NNS
  use/NN
  layers/NNS
  network/NN
  work/NN
  perceptron/NN
  classifier/NN
  network/NN
  activation/NN
  function/NN
  hidden/NN
  layer/NN
  width/NN
  learning/NN
  variation/NN
  number/NN
  layers/NNS
  size/NN
  application/